In [ ]:
import pickle
%matplotlib notebook
import keras
from keras.optimizers.legacy import Adam as LegacyAdam
import numpy as np
from src.utils.preprocessing import load_split_data
from src.utils.visualisation import plot_probability_distributions
import json
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from src.utils.data_io import save_data

In [ ]:
with open("../data/dataset-info-json/subject_to_indices.json", "r") as f:
    subject_to_indices = json.load(f)

subject_to_indices = {int(k): v for k, v in subject_to_indices.items()}
test_subject_id = 1
train_data, train_labels, test_data, test_labels = load_split_data(test_subject_id, subject_to_indices)

In [ ]:
model = keras.models.load_model(f"../models/full_loso/majority_label/model_{test_subject_id}.keras", compile=False)
optimizer = LegacyAdam(learning_rate=1e-3)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
predictions = []
for window in test_data:
    window_reshaped = window.reshape(1,20,6)
    prediction = model.predict(window_reshaped)
    predictions.append(prediction)

In [ ]:
predictions = np.array(predictions)
predictions = predictions.squeeze(axis=1)


In [ ]:
save_data(predictions, "../data/lstm_training_data", f"x_{test_subject_id}")

Plot Predictions

In [ ]:
plot_probability_distributions(predictions, start_index=1655, num_windows=60)

Plot Labels (truth)

In [ ]:
plot_probability_distributions(test_labels, start_index=1655, num_windows=60)

In [ ]:
pred_count = np.sum(predictions[:, 0] > 0.4) # Assign micromovement and threshold probability of choice
true_count = np.sum(test_labels[:, 0]) # Match micromovement

In [ ]:
print(f"Predicted micromovement count: {pred_count}")
print(f"Actual micromovement count: {true_count}")